In [12]:
from __future__ import division
import math
import numpy as np
import numpy.random
import scipy as sp
import scipy.stats
from scipy.optimize import minimize_scalar

# Example of a hybrid audit

Suppose we have two strata, CVR counties and non-CVR counties.
There were 10,000 ballots cast in the election, 90% (9000) in the CVR stratum and 10% (1000) in the non-CVR stratum.

We're interested in a contest between winner $w$ and loser $l$ that appears on 50% of ballots (5000). Suppose the contest was reported exactly correctly: $w$ received 2,600 votes and $\ell$ received 2,400. This is a diluted margin of $2\%$ and a raw margin of 200 ballots.

For $\lambda \in [0,1]$, the outcome of the election was called correctly if the overstatement of the margin in the CVR stratum is less than $200\lambda$ votes and if the overstatement of the margin in the non-CVR stratum is less than $200(1-\lambda)$ votes. For this example, we'll set $\lambda = 0.9$ to reflect the sizes of the two strata.

We'd like to limit the audit risk to $\alpha=10\%$. We divide risk unequally between the two strata: $\alpha_1 = 3\%$ in the CVR stratum and $\alpha_2 = 7\%$ in the non-CVR stratum.

We do this by testing the following null hypotheses:
* in the CVR stratum, test the null that the overstatment is less than $200\lambda$ at risk level $\alpha_1$ using a ballot comparison test
* in the non-CVR stratum, test the null that the overstatment is less than $200(1-\lambda)$ at risk level $\alpha_2$ using a ballot polling test



In [2]:
lambda1 = 0.9
lambda2 = 1-lambda1
alpha1 = 0.03
alpha2 = 0.07
margin = 200
N1 = 9000
N2 = 1000

# CVR stratum

Below, we compute the sample size needed to confirm the election outcome, for varying rates of errors in the population of ballots.

We take the chosen $\lambda$ from above and plug it in as the parameter `null_lambda` in the function below.

We set $\gamma = 1.03905$ as in "A Gentle Introduction to Risk-limiting Audits".

In [5]:
def findNmin_ballot_comparison_rates(alpha, gamma, r1, s1, r2, s2,
                                reported_margin, N, null_lambda=1):

    """
    Compute the smallest sample size for which a ballot comparison 
    audit with the given statistics could stop
    
    Parameters
    ----------
    alpha : float
        risk limit
    gamma : float
        value > 1 to inflate the allowable error
    r1 : int
        rate of ballots that overstate any 
        margin by one vote but no margin by two votes
    s1 : int
        rate of ballots that understate any margin by 
        exactly one vote, and every margin by at least one vote
    r2 : int
        rate of ballots that overstate any margin by two votes
    s2 : int
        rate of ballots that understate every margin by two votes
    reported_margin : float
        the smallest reported margin *in votes* between a winning
        and losing candidate
    N : int
        number of votes cast
    null_lambda : float
        value that describes the null difference between reported and actual votes
        
    Returns
    -------
    n
    """
    m = (reported_margin/null_lambda)/N

    denom = (np.log(1 - m/(2*gamma)) -
                r1*np.log(1 - 1/(2*gamma))- \
                r2*np.log(1 - 1/gamma) - \
                s1*np.log(1 + 1/(2*gamma)) - \
                s2*np.log(1 + 1/gamma) )
    if denom < 0:
        n0 = np.log(alpha)/denom
    else:
        n0 = N
    return int(n0)

In [6]:
# Assuming that the audit will find no errors
findNmin_ballot_comparison_rates(alpha=alpha1, gamma=1.03905, 
                                r1=0, s1=0, r2=0, s2=0,
                                reported_margin=200, N=N1, null_lambda=lambda1)

293

In [8]:
# Assuming that the audit will find 1-vote overstatements at rate 1%
findNmin_ballot_comparison_rates(alpha=alpha1, gamma=1.03905, 
                                r1=0.01, s1=0, r2=0, s2=0,
                                reported_margin=200, N=N1, null_lambda=lambda1)

650

In [9]:
# Assuming that the audit will find 1-vote understatements at rate 1%
findNmin_ballot_comparison_rates(alpha=alpha1, gamma=1.03905, 
                                r1=0, s1=0.01, r2=0, s2=0,
                                reported_margin=200, N=N1, null_lambda=lambda1)

220

# Non-CVR stratum

Below, we compute the sample size needed to confirm the election outcome, for varying sample size $n$.

Define
$$
    c = \text{reported margin in the stratum } - \lambda_2 \text{overall reported margin}.
$$

The reported margin in the stratum could be large or small, but it is known. Below, we will vary it just to see the effect.

$c$ defines the null hypothesis. We test the null that the actual margin in the stratum is less than or equal to $c$: $A_{w, 2} - A_{\ell, 2} \leq c$. Here, $A_{w, 2}$ is an unknown nuisance parameter.

In practice, we will maximize the $p$-value over all possible values of $A_{w,2}$ and $A_{\ell, 2}$ under the null.

In [10]:
def diluted_margin_trihyper_conditional(w, l, n, N_w, N_l, N):
    pvalue = 0
    delta = w-l
    n_wl = w+l
    N_u = N-N_w-N_l
    for ww in range(w, n+1):
        if ww > N_w or (n_wl - ww) > N_l or (n_wl - ww) < 0:
            continue
        pvalue += sp.misc.comb(N_w, ww)*sp.misc.comb(N_l, n_wl - ww)
    return pvalue*sp.misc.comb(N_u, n-n_wl)/sp.misc.comb(N, n)


def trihypergeometric_optim(sample, popsize, null_margin, distr = "conditional"):
    '''
    Wrapper function for p-value calculations using the tri-hypergeometric distribution.
    
    distr = "tri-hypergeometric" or "conditional"
    '''
    
    w = sum(sample==1)
    l = sum(sample==0)
    n = len(sample)
    u = n-w-l    

    # maximize p-value over N_wl
    if distr == "tri-hypergeometric":
        optim_fun = lambda N_w: -1*diluted_margin_trihypergeometric(w, l, n, N_w, N_w-null_margin, popsize)
        upper_WL_limit = (popsize-u+null_margin)/2
        lower_WL_limit = w
    
        res = minimize_scalar(optim_fun, 
                          bounds = [lower_WL_limit, upper_WL_limit], 
                          method = 'bounded')
    elif distr == "conditional":
        optim_fun = lambda N_w: -1*diluted_margin_trihyper_conditional(w, l, n, N_w, N_w-null_margin, popsize)
        upper_WL_limit = (popsize-u+null_margin)/2
        lower_WL_limit = w
    
        res = minimize_scalar(optim_fun, 
                          bounds = [lower_WL_limit, upper_WL_limit], 
                          method = 'bounded')
    else:
        raise ValueError("bad distr arg")
    pvalue = -1*res['fun']
    return pvalue

In [13]:
# Assuming that the audit finds exactly the right vote proportions
# and that the stratum reported margin (in percentages) equals the overall
# sample size = 100, 10% of the stratum size.

sample = np.array([0]*24 + [1]*26 + [np.nan]*50)
c = 0.02*N2 - lambda2*margin
trihypergeometric_optim(sample, popsize=N2, null_margin=c, distr="conditional")

0.036982082644177891

In [16]:
# Assuming that the audit finds exactly the right vote proportions
# and that the stratum reported margin is 1% -- less than the overall
# sample size = 100, 10% of the stratum size.

sample = np.array([0]*24 + [1]*26 + [np.nan]*50)
c = 0.01*N2 - lambda2*margin
trihypergeometric_optim(sample, popsize=N2, null_margin=c, distr="conditional")

0.032104460018494638

In [17]:
# Assuming that the audit finds exactly the right vote proportions
# and that the stratum reported margin is 5% -- greater than the overall
# sample size = 100, 10% of the stratum size.

sample = np.array([0]*24 + [1]*26 + [np.nan]*50)
c = 0.05*N2 - lambda2*margin
trihypergeometric_optim(sample, popsize=N2, null_margin=c, distr="conditional")

0.051873249394131002